# Learning from neighbours

_Figure 2, page 58, from the book "Misinformation Age" by O'Connor & Weatherall_

In [ ]:
# Enable auto-reloading
%reload_ext autoreload
# 2: Reload all
%autoreload 2

import torch

import polygraphs as pg

from polygraphs import hyperparameters as hparams
from polygraphs import ops
from polygraphs import graphs


# When the mistruct multiplier is 0, `OConnorWeatherallOp` is equivalent to `BalaGoyalOp`.
_PARENT_OP = ops.OConnorWeatherallOp


class SimpleOp(_PARENT_OP):
    """
    Overides local signal observed by each agent in the network.
    """
    def sample(self):
        return torch.Tensor([6, 5, 7, 4, 2, 5])

    def trials(self):
        return torch.full((6,), 10)


# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are hard-coded
params.init.kind = 'constant'
params.init.constant.value = [0.32, 0.21, 0.56, 0.43, 0.02, 0.59]
# Chance that action B is better than action A
params.epsilon = 0.1
# Consider sample network with 6 agents
params.network.kind = 'sample'

# Create a DGL graph with given configuration
graph = graphs.create(params.network)
# Create a model with given configuration
model = SimpleOp(graph, params)


result = model(graph)

# Run a single simulation step
_ = pg.simulate_(graph, model, steps=1)

# Print agent beliefs after 1 step
print('New beliefs are', graph.ndata['beliefs'].numpy().round(2))
print('Bye.')